# Importing Libraries

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from scipy import stats
# from sklearn.neighbors import KNeighborsClassifier
#from sklearn.cluster import KMeans
# from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
#import sklearn as skl

# from sklearn.metrics.pairwise import cosine_similarity
# from scipy import sparse

# import tensorflow as tf

# Loading the Data

In [3]:
df_bg_tmp = pd.read_csv('/kaggle/input/boardgame-cleaned/games-cleaned.csv')
df_rv_tmp = pd.read_csv('/kaggle/input/boardgame-cleaned/reviews-cleaned.csv')

# Content-based Recommenders

In [4]:
df_bg = df_bg_tmp.copy()
df_rv = df_rv_tmp.copy()

In [17]:
df_bg.head()

,id,name,rank,geek_rating,avg_rating,usersrated,category,mechanic,weight,minplayers,maxplayers,playingtime,minage,owned,yearpublished,designer,artist,publisher,description,image
0,30549,Pandemic,106,7.48669,7.58896,109006,['Medical'],"['Action Points', 'Cooperative Game', 'Hand Ma...",2.4063,2,4,45,8,168364,2008,['Matt Leacock'],"['Josh Cappel', 'Christian Hanisch', 'Régis Mo...","['Z-Man Games', 'Albi', 'Asmodee', 'Asmodee It...","In Pandemic, several virulent diseases have br...",https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...
1,822,Carcassonne,191,7.30857,7.41837,108776,"['City Building', 'Medieval', 'Territory Build...","['Area Majority / Influence', 'Map Addition', ...",1.9057,2,5,45,7,161299,2000,['Klaus-Jürgen Wrede'],"['Doris Matthäus', 'Anne Pätzke', 'Chris Quill...","['Hans im Glück', '999 Games', 'Albi', 'Bard C...",Carcassonne is a tile-placement game in which ...,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...
2,13,Catan,429,6.96965,7.13598,108064,"['Economic', 'Negotiation']","['Dice Rolling', 'Hexagon Grid', 'Income', 'Mo...",2.3130,3,4,120,10,167733,1995,['Klaus Teuber'],"['Volkan Baga', 'Tanja Donner', 'Pete Fenlon',...","['KOSMOS', '999 Games', 'Albi', 'Asmodee', 'As...","In CATAN (formerly The Settlers of Catan), pla...",https://cf.geekdo-images.com/W3Bsga_uLP9kO91gZ...
3,68448,7 Wonders,73,7.63355,7.73515,90021,"['Ancient', 'Card Game', 'City Building', 'Civ...","['Drafting', 'Hand Management', 'Set Collectio...",2.3264,2,7,30,10,120466,2010,['Antoine Bauza'],"['Dimitri Chappuis', 'Miguel Coimbra', 'Etienn...","['Repos Production', 'ADC Blackfire Entertainm...",You are the leader of one of the 7 great citie...,https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0I...
4,36218,Dominion,104,7.49912,7.61000,81582,"['Card Game', 'Medieval']","['Deck, Bag, and Pool Building', 'Delayed Purc...",2.3542,2,4,30,13,106956,2008,['Donald X. Vaccarino'],"['Matthias Catrein', 'Julien Delval', 'Tomasz ...","['Rio Grande Games', '999 Games', 'Albi', 'Bar...","&quot;You are a monarch, like your parents bef...",https://cf.geekdo-images.com/j6iQpZ4XkemZP07HN...


In [20]:
df_rv.head()

,user,name,rating,comment
0,Torsten,Pandemic,10.0,NaN
1,avlawn,Pandemic,10.0,I tend to either love or easily tire of co-op ...
2,Mike Mayer,Pandemic,10.0,NaN
3,cfarrell,Pandemic,10.0,Hey! I can finally rate this game I've been pl...
4,PopeBrain,Pandemic,10.0,(25+ plays) Who can play? Family game for ...


In [5]:
df_bg_cat = df_bg[['name', 'category', 'mechanic', 'designer', 'artist', 'publisher']].copy()

df_bg_cat = df_bg_cat.set_index('name')
df_bg_cat.head()

,category,mechanic,designer,artist,publisher
name,,,,,
Pandemic,['Medical'],"['Action Points', 'Cooperative Game', 'Hand Ma...",['Matt Leacock'],"['Josh Cappel', 'Christian Hanisch', 'Régis Mo...","['Z-Man Games', 'Albi', 'Asmodee', 'Asmodee It..."
Carcassonne,"['City Building', 'Medieval', 'Territory Build...","['Area Majority / Influence', 'Map Addition', ...",['Klaus-Jürgen Wrede'],"['Doris Matthäus', 'Anne Pätzke', 'Chris Quill...","['Hans im Glück', '999 Games', 'Albi', 'Bard C..."
Catan,"['Economic', 'Negotiation']","['Dice Rolling', 'Hexagon Grid', 'Income', 'Mo...",['Klaus Teuber'],"['Volkan Baga', 'Tanja Donner', 'Pete Fenlon',...","['KOSMOS', '999 Games', 'Albi', 'Asmodee', 'As..."
7 Wonders,"['Ancient', 'Card Game', 'City Building', 'Civ...","['Drafting', 'Hand Management', 'Set Collectio...",['Antoine Bauza'],"['Dimitri Chappuis', 'Miguel Coimbra', 'Etienn...","['Repos Production', 'ADC Blackfire Entertainm..."
Dominion,"['Card Game', 'Medieval']","['Deck, Bag, and Pool Building', 'Delayed Purc...",['Donald X. Vaccarino'],"['Matthias Catrein', 'Julien Delval', 'Tomasz ...","['Rio Grande Games', '999 Games', 'Albi', 'Bar..."


In [6]:
for col in df_bg_cat.columns:
    df_dummy = pd.concat([df_bg_cat[col].str.get_dummies(sep=",")])
df_dummy.head()

,"""Don't Panic Games""","""Don't Panic Games""]","""Dr. Finn's Games""","""Gam'inBIZ""","""Game's Up""]","""Grandpa Beck's Games""","""Kellogg's""","""Lock 'n Load Publishing","""Mad Man's Magic""","""OPEN'N PLAY""",...,['hobbity.eu',['itten'],['moses. Verlag GmbH',['moses. Verlag GmbH'],['subQuark,['wiseandotherwise.com,['こっち屋 (Kocchiya)',['ボドゲイム (Bodogeimu)',['新ボードゲーム党 (New Board Game Party)',['桜遊庵 (Ouyuuan)'
name,,,,,,,,,,,,,,,,,,,,,
Pandemic,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Carcassonne,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Catan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7 Wonders,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dominion,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
content_sim_df = pd.DataFrame(cosine_similarity(df_dummy), columns=df_dummy.index, index=df_dummy.index)
content_sim_df.head()

name,Pandemic,Carcassonne,Catan,7 Wonders,Dominion,Ticket to Ride,Codenames,Terraforming Mars,7 Wonders Duel,Agricola,...,RRR,Magic: The Gathering – Heroes of Dominaria Board Game,"Gulf, Mobile & Ohio",1805: Sea of Glory,Steam Donkey,Feierabend,Nations at War: White Star Rising,Oaxaca: Crafts of a Culture,A Gentle Rain,Wizards of the Wild
name,,,,,,,,,,,,,,,,,,,,,
Pandemic,1.000000,0.264820,0.251259,0.465242,0.290382,0.295540,0.312653,0.174078,0.399362,0.359425,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Carcassonne,0.264820,1.000000,0.463553,0.180702,0.458190,0.204980,0.242871,0.202837,0.155113,0.387783,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Catan,0.251259,0.463553,1.000000,0.154303,0.331061,0.175035,0.181467,0.115470,0.099340,0.264906,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7 Wonders,0.465242,0.180702,0.154303,1.000000,0.278639,0.324102,0.240008,0.213809,0.613139,0.122628,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dominion,0.290382,0.458190,0.331061,0.278639,1.000000,0.151717,0.149801,0.083406,0.191346,0.382692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Top 20 similar board games
content_input = 'Wingspan'
content_sim = content_sim_df[content_input]
content_sim = content_sim.sort_values(ascending=False).head(20)
content_sim

name
Wingspan                                  1.000000
Tapestry                                  0.576557
Viticulture Essential Edition             0.576557
Charterstone                              0.543928
Between Two Castles of Mad King Ludwig    0.532181
Terraforming Mars                         0.470679
Scythe                                    0.449921
Red Rising                                0.380750
Pendulum                                  0.372104
Patchwork                                 0.364179
The Quest for El Dorado                   0.339683
My Little Scythe                          0.339683
Rolling Realms                            0.320256
7 Wonders Duel                            0.314945
Citadels                                  0.314945
The Quacks of Quedlinburg                 0.314485
Power Grid                                0.299572
PARKS                                     0.296500
Pictures                                  0.295656
Port Royal                

In [9]:
df_bg_cat.loc[content_sim.index[:4]]

,category,mechanic,designer,artist,publisher
name,,,,,
Wingspan,"['Animals', 'Card Game', 'Educational']","['Card Drafting', 'Dice Rolling', 'Drafting', ...",['Elizabeth Hargrave'],"['Ana Maria Martinez Jaramillo', 'Natalia Roja...","['Stonemaier Games', '999 Games', 'Angry Lion ..."
Tapestry,['Civilization'],"['Area Majority / Influence', 'Card Drafting',...",['Jamey Stegmaier'],"['Andrew Bosley', 'Rom Brown']","['Stonemaier Games', '999 Games', 'Albi', 'Ang..."
Viticulture Essential Edition,"['Economic', 'Farming']","['Contracts', 'Hand Management', 'Solo / Solit...","['Jamey Stegmaier', 'Alan Stone']","['Jacqui Davis', 'David Montgomery', 'Beth Sob...","['Stonemaier Games', 'Angry Lion Games', 'Arcl..."
Charterstone,"['City Building', 'Economic', 'Medieval']","['Card Drafting', 'Catch the Leader', 'Hand Ma...",['Jamey Stegmaier'],"['Lina Cossette', 'David Forest']","['Stonemaier Games', 'Albi', 'Chronicle Games ..."


In [10]:
df_rv_rate = df_rv.set_index('user').copy()
df_rv_rate = df_rv_rate.groupby(['user', 'name']).mean()
df_rv_rate.head()

rating
user            name                     
1 Family Meeple 10 Days in Europe     4.1
                12 Days               7.0
                7 Wonders             6.5
                A Column of Fire      5.0
                A Feast for Odin     10.0

In [16]:
df_rv_rate.loc["Torsten"]

,rating
name,
11 nimmt!,7.0
1960: The Making of the President,8.5
2 de Mayo,7.0
23,7.5
4 Gods,6.0
...,...
Zombie Kidz Evolution,8.5
Zombie Teenz Evolution,8.0
Zombie in My Pocket,5.0
